In [1]:
""" DenseNet Model for cifar10 """

# Import libraries
import numpy as np
import tensorflow as tf
import keras
import keras.backend as K 
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.models import Sequential
from keras.optimizers import Optimizer
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

import densenet

# For reproducibility
np.random.seed(42)
tf.set_random_seed(42)

Using TensorFlow backend.


In [2]:
# data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [3]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [4]:
# Swish activation function
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)

# Custom activation function 1
# mix between relu and positive part of swish mirrored across x=1
def e_swish_1(x):
    return K.maximum(0.0, x*(2-K.sigmoid(x)))

# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [5]:
activations = [e_swish_2, "relu", swish, e_swish_1]
names = activations[:]
for i,a in enumerate(names):
    if not isinstance(a, str):
        names[i] = a.__name__
print(names)

['e_swish_2', 'relu', 'swish', 'e_swish_1']


In [6]:
###################
# Construct model #
###################

act = activations[0]
depth = 40
nb_dense_block = 3
nb_filter = 16 
growth_rate = 12
dropout_rate = 0.0
weight_decay = 1e-4

model = densenet.DenseNet(
                          act, 10,
                          (32, 32, 3),
                          depth,
                          nb_dense_block,
                          growth_rate,
                          nb_filter,
                          dropout_rate=dropout_rate,
                          weight_decay=weight_decay
                         )

# Model output
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
initial_conv2D (Conv2D)         (None, 32, 32, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   128         initial_conv2D[0][0]             
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_1 (

In [10]:
####################
# Network training #
####################
batch_size  = 64
epochs = 30

# Build optimizer
opt = keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
                     
print("Training up to 50%")
for i in range(5):
    his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
    model.save_weights('densenet_00_'+str(i)+'.h5')
    print(his.history)

Training up to 50%
Epoch 1/30


ResourceExhaustedError: OOM when allocating tensor with shape[64,32,32,148]
	 [[Node: batch_normalization_12/moments/SquaredDifference = SquaredDifference[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](concatenate_11/concat, batch_normalization_12/moments/mean)]]
	 [[Node: dense_1/BiasAdd/_1263 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_8478_dense_1/BiasAdd", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'batch_normalization_12/moments/SquaredDifference', defined at:
  File "c:\users\eric\appdata\local\programs\python\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\eric\appdata\local\programs\python\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\eric\appdata\local\progr

In [10]:
####################
# Network training #
####################
batch_size  = 64
epochs = 25

# Build optimizer
opt = keras.optimizers.SGD(lr=0.1/10, momentum=0.9, decay=1e-4, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
                     
print("Training up to 75%")
for i in range(3):
    his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
    model.save_weights('densenet_50_'+str(i)+'.h5')
    print(his.history)

Training
Epoch 1/1
782/781 [==============================] - 1417s 2s/step - loss: 1.8527 - acc: 0.3136 - val_loss: 1.5694 - val_acc: 0.4225


In [12]:
####################
# Network training #
####################
batch_size  = 64
epochs = 25

# Build optimizer
opt = keras.optimizers.SGD(lr=0.1/100, momentum=0.9, decay=1e-4, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
                     
print("Training up to 100%")
for i in range(3):
    his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
    model.save_weights('densenet_75_'+str(i)+'.h5')
    print(his.history)

Training
Epoch 1/1
782/781 [==============================] - 1395s 2s/step - loss: 1.3601 - acc: 0.5043 - val_loss: 1.3398 - val_acc: 0.5109
Training
Epoch 1/1
782/781 [==============================] - 1391s 2s/step - loss: 1.2732 - acc: 0.5371 - val_loss: 1.1861 - val_acc: 0.5660
Training
Epoch 1/1
782/781 [==============================] - 1391s 2s/step - loss: 1.2062 - acc: 0.5637 - val_loss: 1.0978 - val_acc: 0.6060
Training
Epoch 1/1
782/781 [==============================] - 1391s 2s/step - loss: 1.1450 - acc: 0.5877 - val_loss: 0.9926 - val_acc: 0.6454
Training
Epoch 1/1
782/781 [==============================] - 1391s 2s/step - loss: 1.0974 - acc: 0.6074 - val_loss: 0.9943 - val_acc: 0.6384
Training
Epoch 1/1
782/781 [==============================] - 1391s 2s/step - loss: 1.0569 - acc: 0.6227 - val_loss: 0.9442 - val_acc: 0.6559
Training
Epoch 1/1
782/781 [==============================] - 1391s 2s/step - loss: 1.0239 - acc: 0.6359 - val_loss: 0.9114 - val_acc: 0.6700
Traini